In [1]:
!pip install openai langchain faiss-cpu


In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os


In [3]:
sample_text = """
Oracle compliance involves ensuring user access to data is tracked and aligned with audit policies.
Audit logs must be retained for 7 years. Access to sensitive columns must be masked.
"""
with open("oracle_compliance.txt", "w") as f:
    f.write(sample_text)


In [4]:
loader = TextLoader("oracle_compliance.txt")
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs_split = splitter.split_documents(docs)


In [6]:
from langchain.llms.base import LLM
from typing import List, Optional
from pydantic import BaseModel

class DummyLLM(LLM, BaseModel):
    response: str = "Simulated answer from AI model based on retrieved chunks."

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self.response

    @property
    def _identifying_params(self):
        return {}

    @property
    def _llm_type(self):
        return "dummy"



In [8]:
import numpy as np


In [9]:
from langchain.embeddings.base import Embeddings

class DummyEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return [np.random.rand(1536).tolist() for _ in texts]

    def embed_query(self, text):
        return np.random.rand(1536).tolist()


In [10]:
embeddings = DummyEmbeddings()
db = FAISS.from_documents(docs_split, embeddings)


In [13]:
import numpy as np
from langchain.embeddings.base import Embeddings

class DummyEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return [np.random.rand(1536).tolist() for _ in texts]

    def embed_query(self, text):
        return np.random.rand(1536).tolist()

embeddings = DummyEmbeddings()
db = FAISS.from_documents(docs_split, embeddings)

llm = DummyLLM()

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

query = "What is the retention policy for Oracle audit logs?"
response = qa.run(query)

print("Simulated AI Answer:", response)


Simulated AI Answer: Simulated answer from AI model based on retrieved chunks.
